In [1]:
exec('from __future__ import absolute_import, division, print_function, unicode_literals')

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras import layers

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown
%matplotlib inline

## Reading in and Preprocessing Data

In [2]:
train = pd.read_csv('aps_failure_training_set_processed_8bit.csv')
test = pd.read_csv('aps_failure_test_set_processed_8bit.csv')

In [3]:
test_set = test.copy()

train_copy = train.copy()
train_set = train_copy.sample(frac=0.9, random_state = 0)
val_set = train_copy.drop(train_set.index)

In [4]:
train_set.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
3048,-0.992188,0.015625,-0.289062,0.992188,-0.007812,-0.046875,-0.054688,-0.007812,-0.03125,-0.054688,...,-0.015625,0.015625,-0.000000,0.031250,0.117188,-0.031250,0.414062,0.085938,-0.023438,-0.023438
19563,-0.992188,0.992188,-0.289062,0.992188,-0.007812,-0.046875,-0.054688,-0.007812,-0.03125,-0.054688,...,0.992188,0.992188,0.992188,0.992188,-0.187500,-0.195312,-0.304688,-0.171875,0.406250,0.375000
58303,-0.992188,-0.398438,0.554688,-0.468750,-0.007812,-0.046875,-0.054688,-0.007812,-0.03125,-0.054688,...,-0.375000,-0.382812,-0.375000,-0.343750,-0.304688,-0.187500,-0.304688,-0.171875,-0.023438,-0.023438
8870,-0.992188,0.992188,0.640625,-0.468750,-0.007812,-0.046875,-0.054688,-0.007812,-0.03125,-0.046875,...,0.992188,0.992188,0.992188,0.539062,0.289062,0.460938,-0.257812,-0.171875,-0.023438,-0.023438
40228,-0.992188,0.015625,-0.289062,-0.468750,-0.007812,-0.046875,-0.054688,-0.007812,-0.03125,-0.054688,...,0.445312,0.453125,0.226562,0.007812,-0.132812,-0.148438,-0.203125,-0.171875,-0.023438,-0.023438


In [5]:
train_set['class'] = (train_set['class'] > 0).astype(int)
val_set['class'] = (val_set['class'] > 0).astype(int)
test_set['class'] = (test_set['class'] > 0).astype(int)

In [6]:
# SMOTE TO Balance Dataset

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE 


print('Original dataset shape %s' % Counter(train_set['class'].to_numpy()))
sm = SMOTE(random_state=42)

X_res, y_res = sm.fit_resample(train_set.to_numpy(), train_set['class'].to_numpy())
print('Resampled dataset shape %s' % Counter(y_res))

print('Original dataset shape %s' % Counter(val_set['class'].to_numpy()))
sm = SMOTE(random_state=42)
X_val_res, y_val_res = sm.fit_resample(val_set.to_numpy(), val_set['class'].to_numpy())
print('Resampled dataset shape %s' % Counter(y_val_res))


Using TensorFlow backend.


Original dataset shape Counter({0: 53092, 1: 908})
Resampled dataset shape Counter({0: 53092, 1: 53092})
Original dataset shape Counter({0: 5908, 1: 92})
Resampled dataset shape Counter({0: 5908, 1: 5908})


In [7]:
y_res_pd = pd.Series(y_res)
y_train = pd.get_dummies(y_res_pd)

y_val_res_pd = pd.Series(y_val_res)
y_val = pd.get_dummies(y_val_res_pd)

train_set = pd.DataFrame(X_res, columns=list(train_set.columns.values))
val_set = pd.DataFrame(X_val_res, columns=list(val_set.columns.values))

In [8]:
y_test = pd.get_dummies(test_set['class'])



train_set.drop(columns=['class'],inplace=True)
val_set.drop(columns=['class'],inplace=True)
test_set.drop(columns=['class'],inplace=True)

In [9]:
y_train.describe()

,0,1
count,106184.000000,106184.000000
mean,0.500000,0.500000
std,0.500002,0.500002
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.500000,0.500000
75%,1.000000,1.000000
max,1.000000,1.000000


In [10]:
train_set_np = train_set.to_numpy()
y_train_np = y_train.to_numpy().astype('float32')
val_set_np = val_set.to_numpy()
y_val_np = y_val.to_numpy().astype('float32')
test_set_np = test_set.to_numpy()
y_test_np = y_test.to_numpy().astype('float32')

In [11]:
y_test_np.shape

(16000, 2)

In [12]:
# Reshape for CNN1D 
train_set_cnn = train_set_np.reshape(106184, 170, 1)
val_set_cnn = val_set_np.reshape(11816, 170, 1)
test_set_cnn = test_set_np.reshape(16000, 170, 1)

## Focal Loss Implementation 

In [13]:
# Focal Loss function

class FocalLoss(keras.losses.Loss):
    def __init__(self, gamma=5., alpha=.75,
                 reduction=keras.losses.Reduction.AUTO, name='focal_loss'):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})
        """
        super(FocalLoss, self).__init__(reduction=reduction,
                                        name=name)
        self.gamma = float(gamma)
        self.alpha = float(alpha)

    def call(self, y_true, y_pred):
        """
        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(
            tf.subtract(1., model_out), self.gamma))
        fl = tf.multiply(self.alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)

## Define Models (Focal Loss vs Cross Entropy) 

In [14]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]
###############################################################################################################
# *************************************************************************************************************
###############################################################################################################

model_cnn_fl = tf.keras.Sequential()
model_cnn_fl.add(layers.Conv1D(10, 1, activation='relu', input_shape=(170, 1),data_format='channels_last'))
model_cnn_fl.add(layers.Flatten())
model_cnn_fl.add(layers.Dense(64, activation='relu', bias_initializer=tf.constant_initializer(0.01),
                              kernel_regularizer=regularizers.l2(0.0001)))
model_cnn_fl.add(layers.Dropout(0.5))
model_cnn_fl.add(layers.Dense(2, activation='softmax'))

model_cnn_fl.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=FocalLoss(),
              metrics=METRICS)

###############################################################################################################
# *************************************************************************************************************
###############################################################################################################

model_cnn_bc = tf.keras.Sequential()
model_cnn_bc.add(layers.Conv1D(10, 1, activation='relu', input_shape=(170, 1),data_format='channels_last'))
model_cnn_bc.add(layers.Flatten())
model_cnn_bc.add(layers.Dense(64, activation='relu', bias_initializer=tf.constant_initializer(0.01),
                              kernel_regularizer=regularizers.l2(0.0001)))
model_cnn_bc.add(layers.Dropout(0.5))
model_cnn_bc.add(layers.Dense(2, activation='softmax'))

model_cnn_bc.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='binary_crossentropy',
              metrics=METRICS)

## Training Models 

In [15]:
model_cnn_fl.fit(train_set_cnn,y_train_np, batch_size=256, epochs=20, validation_data=(val_set_cnn,y_val_np))

Train on 106184 samples, validate on 11816 samples
Epoch 1/20
106184/106184 [==============================] - 7s 70us/sample - loss: 0.0124 - tp: 99931.0000 - fp: 6253.0000 - tn: 99931.0000 - fn: 6253.0000 - accuracy: 0.9411 - precision: 0.9411 - recall: 0.9411 - auc: 0.9773 - val_loss: 0.0046 - val_tp: 11427.0000 - val_fp: 389.0000 - val_tn: 11427.0000 - val_fn: 389.0000 - val_accuracy: 0.9671 - val_precision: 0.9671 - val_recall: 0.9671 - val_auc: 0.9949
Epoch 2/20
106184/106184 [==============================] - 5s 47us/sample - loss: 0.0059 - tp: 100325.0000 - fp: 5859.0000 - tn: 100325.0000 - fn: 5859.0000 - accuracy: 0.9448 - precision: 0.9448 - recall: 0.9448 - auc: 0.9837 - val_loss: 0.0047 - val_tp: 11437.0000 - val_fp: 379.0000 - val_tn: 11437.0000 - val_fn: 379.0000 - val_accuracy: 0.9679 - val_precision: 0.9679 - val_recall: 0.9679 - val_auc: 0.9951
Epoch 3/20
106184/106184 [==============================] - 6s 52us/sample - loss: 0.0064 - tp: 99931.0000 - fp: 6253.0000 - 

In [16]:
model_cnn_bc.fit(train_set_cnn,y_train_np, batch_size=256, epochs=20, validation_data=(val_set_cnn,y_val_np))

Train on 106184 samples, validate on 11816 samples
Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
106184/106184 [==============================] - 7s 70us/sample - loss: 0.1554 - tp: 101638.0000 - fp: 4546.0000 - tn: 101638.0000 - fn: 4546.0000 - accuracy: 0.9572 - precision: 0.9572 - recall: 0.9572 - auc: 0.9880 - val_loss: 0.1193 - val_tp: 11475.0000 - val_fp: 341.0000 - val_tn: 11475.0000 - val_fn: 341.0000 - val_accuracy: 0.9711 - val_precision: 0.9711 - val_recall: 0.9711 - val_auc: 0.9932
Epoch 2/20
106184/106184 [==============================] - 6s 54us/sample - loss: 0.1313 - tp: 102404.0000 - fp: 3780.0000 - tn: 102404.0000 - fn: 3780.0000 - accuracy: 0.9644 - precision: 0.9644 - recall: 0.9644 - auc: 0.9922 - val_loss: 0.0979 - val_tp: 11535.0000 - val_fp: 281.0000 - val_tn: 11535.0000 - val_fn: 281.0000 - val_accuracy: 0.9762 - val_precision: 0.9762 - val_recall: 0.9762 - val_auc: 0.9961
Epoch 3/20
106184/106184 [===

106184/106184 [==============================] - 7s 63us/sample - loss: 0.1141 - tp: 103417.0000 - fp: 2767.0000 - tn: 103417.0000 - fn: 2767.0000 - accuracy: 0.9739 - precision: 0.9739 - recall: 0.9739 - auc: 0.9939 - val_loss: 0.0917 - val_tp: 11578.0000 - val_fp: 238.0000 - val_tn: 11578.0000 - val_fn: 238.0000 - val_accuracy: 0.9799 - val_precision: 0.9799 - val_recall: 0.9799 - val_auc: 0.9970
Epoch 20/20
106184/106184 [==============================] - 5s 45us/sample - loss: 0.1181 - tp: 103288.0000 - fp: 2896.0000 - tn: 103288.0000 - fn: 2896.0000 - accuracy: 0.9727 - precision: 0.9727 - recall: 0.9727 - auc: 0.9931 - val_loss: 0.0950 - val_tp: 11563.0000 - val_fp: 253.0000 - val_tn: 11563.0000 - val_fn: 253.0000 - val_accuracy: 0.9786 - val_precision: 0.9786 - val_recall: 0.9786 - val_auc: 0.9975


## Results 

In [17]:
def final_cost(result, loss_name):
    Cost_1 = 10
    Cost_2 = 500
    tn, fp, fn, tp = confusion_matrix(y_test_ori.to_numpy(), result.to_numpy()).ravel()
    cost = Cost_1*fp + Cost_2*fn
    print('tn: ',tn, 'fp: ',fp,'fn: ',fn,'tp: ',tp)
    print('Final Cost using ',loss_name,' is: ',cost)
    print('Total number of misclassificactions: ', fp+fn)

In [20]:
from sklearn.metrics import confusion_matrix

result_cnn_fl = model_cnn_fl.predict(test_set_cnn)
result_cnn_bc = model_cnn_bc.predict(test_set_cnn)
# print('FL CNN1D Output Shape:', result_cnn_fl.shape)
# print('BC CNN1D Output Shape:', result_cnn_bc.shape)

assert result_cnn_fl.shape == result_cnn_bc.shape

result_cnn_fl_pd = pd.DataFrame(np.rint(result_cnn_fl))
result_cnn_fl_stack_pd = result_cnn_fl_pd.stack()
result_cnn_fl_ori = pd.Series(pd.Categorical(result_cnn_fl_stack_pd[result_cnn_fl_stack_pd!=0].index.get_level_values(1)))

result_cnn_bc_pd = pd.DataFrame(np.rint(result_cnn_bc))
result_cnn_bc_stack_pd = result_cnn_bc_pd.stack()
result_cnn_bc_ori = pd.Series(pd.Categorical(result_cnn_bc_stack_pd[result_cnn_bc_stack_pd!=0].index.get_level_values(1)))

y_test_stack = y_test.stack()
y_test_ori = pd.Series(pd.Categorical(y_test_stack[y_test_stack!=0].index.get_level_values(1)))

print('FL: ',confusion_matrix(y_test_ori.to_numpy(),result_cnn_fl_ori.to_numpy()))
print('BC: ',confusion_matrix(y_test_ori.to_numpy(),result_cnn_bc_ori.to_numpy()))

final_cost(result_cnn_fl_ori, 'Focal Loss - CNN1D')
final_cost(result_cnn_bc_ori, 'Binary Cross Entropy - CNN1D')

FL:  [[14311  1314]
 [    5   370]]
BC:  [[15272   353]
 [   32   343]]
tn:  14311 fp:  1314 fn:  5 tp:  370
Final Cost using  Focal Loss - CNN1D  is:  15640
Total number of misclassificactions:  1319
tn:  15272 fp:  353 fn:  32 tp:  343
Final Cost using  Binary Cross Entropy - CNN1D  is:  19530
Total number of misclassificactions:  385
